# Exective summary of Work Package 3

## Objectives

In this WP, you will work on a given training dataset. Your goal is to develop a fault detection model using the regression algorithms learnt in the class, in order to achieve best F1 score。

## Tasks

- Task 1: Develop a regression model to predict the reference value for motor temperature.
- Task 2: Develop a fault detection model using the regression model you developed in Task 1.

## Delierables

- A Jupyter notebook reporting the process and results of the above tasks


# Before starting, please:
- Fetch the most up-to-date version of the github repository.
- Create a new branch with your name, based on the "main" branch and switch to your own branch.
- Copy this notebook to the work space of your group, and rename it to TD_WP_3_Your name.ipynb
- After finishing this task, push your changes to the github repository of your group.

# Task 1: Predict normal behaviors through regression models

In this task, let us try to develop a best regression model to predict the normal behaviors of a given motor. In this exercise, we can use motor 6 as an example. You can easilily generate the approach to other models for the data challenge.

We can use all the dataset where motor 6 works normally as our dataset. Then, we can run a cross validation (based on sequence, not points) to test the performances of the developed model.

In this example, we mainly use the following performance metrics:
- max error: The max error between the predicted and the true values.
- Mean root squared error: The mean root squared error between the predicted and the true values.
- Out-of-boundary rate: The percentage that the residual error between the predicted and the true values is larger than a given threshold. Here, we set the thresold to be $3$ degree.

## Sub-task 1: Only use the features at the current moment.

[This notebook](demo_regression_mdl.ipynb) provides a basic demonstration of how to set up the experiment. Let us start by considering only using the features from the current moment. In the notebook, we show a baseline using a simple linear regression with all the features. Could you please try to improve the performance of the model?

A few possible directions:
- Feature selection?
- Smoothing?
- Removing sequence-to-sequence variablity? Adding features regarding time dynamics (see the TD for last lecture).
- Changing to other regression models? For this, you can try different regression models from [here](https://scikit-learn.org/stable/supervised_learning.html)

Put your code below.

Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer,accuracy_score ,precision_score, recall_score
from sklearn.metrics import f1_score,confusion_matrix, roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import warnings
from scipy.signal import butter, filtfilt

from functions import *


from utility import read_all_csvs_one_test
from utility import read_all_test_data_from_path
from utility import read_all_test_data_from_path, show_reg_result,extract_selected_feature, prepare_sliding_window, FaultDetectReg

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import f_regression


Read data and pre_processing

In [19]:
n_int = 20

# Subfunction for data preprocessing.
def pre_processing(df: pd.DataFrame):
    ''' ### Description
    Preprocess the data:
    - remove outliers
    - add new features about the difference between the current and previous n data point.
    '''
    
    # Function to design a Butterworth low-pass filter
    def butter_lowpass(cutoff, fs, order=5):
        nyquist = 0.5 * fs
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a


    # Function to apply the Butterworth low-pass filter
    def lowpass_filter(data, cutoff_freq, sampling_freq, order=5):
        b, a = butter_lowpass(cutoff_freq, sampling_freq, order=order)
        filtered_data = filtfilt(b, a, data)
        return filtered_data


    # Set parameters for the low-pass filter
    cutoff_frequency = .8  # Adjust as needed
    sampling_frequency = 10  # Assuming your data is evenly spaced in time


    def customized_outlier_removal(df: pd.DataFrame):
        ''' # Description
        Remove outliers from the dataframe based on defined valid ranges. 
        Define a valid range of temperature and voltage. 
        Use ffil function to replace the invalid measurement with the previous value.
        '''
        df['position'] = df['position'].where(df['position'] <= 1000, np.nan)
        df['position'] = df['position'].where(df['position'] >= 0, np.nan)
        df['position'] = df['position'].ffill()
        df['position'] = lowpass_filter(df['position'], cutoff_frequency, sampling_frequency)
        df['position'] = df['position'].rolling(window=20, min_periods=1).mean()
        df['position'] = df['position'].round()

        df['temperature'] = df['temperature'].where(df['temperature'] <= 100, np.nan)
        df['temperature'] = df['temperature'].where(df['temperature'] >= 0, np.nan)
        df['temperature'] = df['temperature'].rolling(window=20, min_periods=1).mean()

        # Make sure that the difference between the current and previous temperature cannot be too large.
        # Define your threshold
        threshold = 5
        # Shift the 'temperature' column by one row to get the previous temperature
        prev_tmp = df['temperature'].shift(1)
        # Calculate the absolute difference between current and previous temperature
        temp_diff = np.abs(df['temperature'] - prev_tmp)
        # Set the temperature to NaN where the difference is larger than the threshold
        df.loc[temp_diff > threshold, 'temperature'] = np.nan
        df['temperature'] = df['temperature'].ffill()

        df['voltage'] = df['voltage'].where(df['voltage'] >= 6000, np.nan)
        df['voltage'] = df['voltage'].where(df['voltage'] <= 8000, np.nan)
        df['voltage'] = df['voltage'].ffill()
        df['voltage'] = lowpass_filter(df['voltage'], cutoff_frequency, sampling_frequency)
        df['voltage'] = df['voltage'].rolling(window=5, min_periods=1).mean()


    def cal_diff(df: pd.DataFrame, n_int: int):
        ''' # Description
        Calculate the difference between the current and previous n data point.
        '''
        # Tranform the features relative to the first data point.
        df['temperature'] = df['temperature'] - df['temperature'].iloc[0]
        df['voltage'] = df['voltage'] - df['voltage'].iloc[0]
        df['position'] = df['position'] - df['position'].iloc[0]

        # Calculate the difference between the current and previous n data point.
        df['temperature_diff'] = df['temperature'].diff(n_int)
        df['voltage_diff'] = df['voltage'].diff(n_int)
        df['position_diff'] = df['position'].diff(n_int)   

    # Start processing.
    customized_outlier_removal(df)
    #cal_diff(df, n_int)

base_dictionary = '../../dataset/training_data/'
df_data = read_all_test_data_from_path(base_dictionary, pre_processing, is_plot=False)

In [5]:
# Get all the folders in the base_dictionary
path_list = os.listdir(base_dictionary)
# Only keep the folders, not the excel file.
path_list = path_list[:-1]

# Read the data.
df_data_smoothing = pd.DataFrame()
for tmp_path in path_list:
    path = base_dictionary + tmp_path
    # Read the data with the customized outlier removal function.
    tmp_df = read_all_csvs_one_test(path, tmp_path, pre_processing)
    df_data_smoothing = pd.concat([df_data_smoothing, tmp_df])
    df_data_smoothing = df_data_smoothing.reset_index(drop=True)

# Read the test conditions
df_test_conditions = pd.read_excel(base_dictionary+'Test conditions.xlsx')

In [4]:
normal_test_id = ['20240105_164214', 
    '20240105_165300', 
    '20240105_165972', 
    '20240320_152031', 
    '20240320_153841', 
    '20240320_155664', 
    '20240321_122650', 
    '20240325_135213', 
    '20240426_141190', 
    '20240426_141532', 
    '20240426_141602', 
    '20240426_141726', 
    '20240426_141938', 
    '20240426_141980', 
    '20240503_164435']
df_data = df_data[df_data['test_condition'].isin(normal_test_id)]
df_data_smoothing = df_data_smoothing[df_data_smoothing['test_condition'].isin(normal_test_id)]

In [41]:
# Ignore warnings.
warnings.filterwarnings('ignore')

# Steps for Linear Regression
linear_regression_steps = [
    ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
    #('Standardizer', StandardScaler),  # Step 1: Standardization
    ('regressor', LinearRegression())    # Step 2: Linear Regression
]

# Steps for Ridge Regression
ridge_regression_steps = [
    ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
    #('Standardizer', StandardScaler),  # Step 1: Standardization
    ('regressor', Ridge())               # Step 2: Ridge Regression
]

# Steps for Lasso Regression
lasso_regression_steps = [
    ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
    #('Standardizer', StandardScaler),  # Step 1: Standardizatio
    ('regressor', Lasso())               # Step 2: Lasso Regression
]

# Steps for ElasticNet Regression
elasticnet_regression_steps = [
    ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
    #('Standardizer', StandardScaler),  # Step 1: Standardization
    ('regressor', ElasticNet())          # Step 2: ElasticNet Regression
]

# Steps for Decision Tree Regression
decision_tree_steps = [
    ('regressor', DecisionTreeRegressor())  # Step 2: Decision Tree Regressor
]

# Initialize Pipelines for each model
mdl_linear_regression = Pipeline(linear_regression_steps)
mdl_ridge_regression = Pipeline(ridge_regression_steps)
mdl_lasso_regression = Pipeline(lasso_regression_steps)
mdl_elasticnet_regression = Pipeline(elasticnet_regression_steps)
mdl_decision_tree = Pipeline(decision_tree_steps)

# List of models to be used in GridSearchCV
models = [
    ('Linear Regression', mdl_linear_regression),
    ('Ridge Regression', mdl_ridge_regression),
    ('Lasso Regression', mdl_lasso_regression),
    ('ElasticNet Regression', mdl_elasticnet_regression),
    ('Decision Tree Regression', mdl_decision_tree),
]


In [42]:
from utility import run_cv_one_motor


def run_all_motors(df_data, mdl, feature_list, n_fold=5, threshold=3, window_size=1, sample_step=1,
            prediction_lead_time=1, single_run_result=True, mdl_type='reg'):
    all_results = []
    # Loop over all the six motors.
    for i in range(1, 7):
        print(f'Model for predicting temperature of motor {i}:')
        # Run cross validation.
        df_perf = run_cv_one_motor(motor_idx=i, df_data=df_data, mdl=mdl, feature_list=feature_list,
                n_fold=n_fold, threshold=threshold, window_size=window_size, sample_step=sample_step,
            prediction_lead_time=prediction_lead_time, single_run_result=single_run_result, mdl_type=mdl_type)
        all_results.append(df_perf)

    return all_results

In [43]:
def run_all_models_motor6(df_data, models, feature_list, n_fold=5, threshold=3, window_size=1, sample_step=1,
                   prediction_lead_time=1, single_run_result=True, mdl_type='reg'):
    summary_results = []
    for name, mdl in models:
        print(f'Running model: {name}')
        df_perf  = run_cv_one_motor(motor_idx=6, df_data=df_data, mdl=mdl, feature_list=feature_list,
                n_fold=n_fold, threshold=threshold, window_size=window_size, sample_step=sample_step,
            prediction_lead_time=prediction_lead_time, single_run_result=single_run_result, mdl_type=mdl_type)
        
        summary_results.append((name, df_perf))
    
    return summary_results

def run_all_models_motor5(df_data, models, feature_list, n_fold=5, threshold=3, window_size=1, sample_step=1,
                   prediction_lead_time=1, single_run_result=True, mdl_type='reg'):
    summary_results = []
    for name, mdl in models:
        print(f'Running model: {name}')
        df_perf  = run_cv_one_motor(motor_idx=5, df_data=df_data, mdl=mdl, feature_list=feature_list,
                n_fold=n_fold, threshold=threshold, window_size=window_size, sample_step=sample_step,
            prediction_lead_time=prediction_lead_time, single_run_result=single_run_result, mdl_type=mdl_type)
        
        summary_results.append((name, df_perf))
    
    return summary_results


def run_all_models(df_data, models, feature_list, n_fold=5, threshold=3, window_size=1, sample_step=1,
                   prediction_lead_time=1, single_run_result=True, mdl_type='reg'):
    summary_results = []
    for name, mdl in models:
        print(f'Running model: {name}')
        all_results = run_all_motors(df_data, mdl, feature_list, n_fold=n_fold, threshold=threshold, 
                                     window_size=window_size, sample_step=sample_step,
                                     prediction_lead_time=prediction_lead_time, single_run_result=single_run_result, 
                                     mdl_type=mdl_type)
        
        df_all_results = pd.concat(all_results, keys=[f'Motor_{i}' for i in range(1, 7)])
        summary_results.append((name, df_all_results))
    
    return summary_results

Select Features

In [8]:
feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature','data_motor_1_voltage',
       'data_motor_1_temperature_diff', 'data_motor_1_voltage_diff','data_motor_1_position_diff', 
       'data_motor_2_position','data_motor_2_temperature', 'data_motor_2_voltage', 
       'data_motor_2_temperature_diff', 'data_motor_2_voltage_diff', 'data_motor_2_position_diff',
       'data_motor_3_position', 'data_motor_3_temperature','data_motor_3_voltage',
       'data_motor_3_temperature_diff', 'data_motor_3_voltage_diff','data_motor_3_position_diff', 
       'data_motor_4_position','data_motor_4_temperature', 'data_motor_4_voltage',
       'data_motor_4_temperature_diff', 'data_motor_4_voltage_diff', 'data_motor_4_position_diff',
       'data_motor_5_position', 'data_motor_5_temperature','data_motor_5_voltage',
       'data_motor_5_temperature_diff', 'data_motor_5_voltage_diff','data_motor_5_position_diff', 
       'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage',
       'data_motor_6_temperature_diff', 'data_motor_6_voltage_diff', 'data_motor_6_position_diff']


selected_features= ['time','data_motor_1_position',  'data_motor_1_temperature', 
                    'data_motor_2_position',  
                    'data_motor_3_position',  
                    'data_motor_4_position', 'data_motor_4_temperature',
                    'data_motor_5_position', 'data_motor_5_temperature',
                    'data_motor_6_position', 'data_motor_6_temperature', 
                    'data_motor_1_voltage']

threshold = 1
window_size = 1
sample_step = 1
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

#all_model_results = run_all_models(df_data=df_data_smoothing, models=models, feature_list=selected_features, threshold=threshold, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, single_run_result=False)

all_model_results = run_all_models(df_data=df_data_smoothing, models=models, feature_list=selected_features, threshold=threshold, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, single_run_result=False)

Running model: Linear Regression
Model for predicting temperature of motor 1:
Model for motor 1:
   Max error       RMSE  Exceed boundary rate
0  18.561360   8.434151              0.907702
1   9.379152   2.406074              0.807183
2  34.791890  22.018340              0.961836
3  14.205442   5.826287              0.752688
4  23.378524  13.316468              0.991731


Mean performance metric and standard error:
Max error: 20.0633 +- 9.7294
RMSE: 10.4003 +- 7.6187
Exceed boundary rate: 0.8842 +- 0.1017


Model for predicting temperature of motor 2:
Model for motor 2:
   Max error      RMSE  Exceed boundary rate
0   4.587556  1.791137              0.628756
1   8.756402  1.882761              0.323251
2   7.581748  4.428876              0.794304
3   2.304885  0.904167              0.241935
4   3.501288  1.069757              0.294743


Mean performance metric and standard error:
Max error: 5.3464 +- 2.7320
RMSE: 2.0153 +- 1.4161
Exceed boundary rate: 0.4566 +- 0.2417


Model for predi

In [9]:
summary_data = {
    'Model': [],
    'Max error': [],
    'RMSE': [],
    'Exceed boundary rate': []
}

for model_name, df_results in all_model_results:
    max_error = df_results['Max error'].mean()
    mse = df_results['RMSE'].mean()
    exceed_boundary_rate = df_results['Exceed boundary rate'].mean()
    
    summary_data['Model'].append(model_name)
    summary_data['Max error'].append(f'{max_error:.2f}')
    summary_data['RMSE'].append(f'{mse:.2f}')
    summary_data['Exceed boundary rate'].append(f'{exceed_boundary_rate:.2f}')

summary_df = pd.DataFrame(summary_data)

print("Summary of the results - Only using features from the current moment\n")
print("| Model                   | Max error | RMSE  | Exceed boundary rate |")
print("|-------------------------|-----------|-------|----------------------|")

for model_name, df_results in all_model_results:
    max_error = df_results['Max error'].mean()
    mse = df_results['RMSE'].mean()
    exceed_boundary_rate = df_results['Exceed boundary rate'].mean()
    
    print(f"| {model_name.ljust(24)} | {max_error:8.2f} | {mse:6.2f} | {exceed_boundary_rate:6.2f} |")

best_model = summary_df.loc[summary_df['RMSE'].idxmin()]
print(f"\nThe best model is {best_model['Model']} with a Mean Squared Error (RMSE) of {best_model['RMSE']}.")

Summary of the results - Only using features from the current moment

| Model                   | Max error | RMSE  | Exceed boundary rate |
|-------------------------|-----------|-------|----------------------|
| Linear Regression        |    11.34 |   4.77 |   0.68 |
| Ridge Regression         |    11.23 |   4.73 |   0.68 |
| Lasso Regression         |     6.93 |   3.35 |   0.61 |
| ElasticNet Regression    |     6.92 |   3.34 |   0.61 |
| Decision Tree Regression |     8.96 |   3.95 |   0.54 |

The best model is ElasticNet Regression with a Mean Squared Error (RMSE) of 3.34.


**Summary of the results - Only using features from the current moment**

Please add a table in the end, summarying the results from all the models. Please write a few texts to explain what is the best model you got (including the features and preprocessing you did), its performance, and how could you further improve it.

| Model                   | Max error | RMSE  | Exceed boundary rate |
|-------------------------|-----------|-------|----------------------|
| Linear Regression        |     9.92 |   4.56 |   0.49 |
| Ridge Regression         |     9.83 |   4.52 |   0.49 |
| Lasso Regression         |     5.85 |   2.15 |   0.23 |
| ElasticNet Regression    |     5.85 |   2.15 |   0.23 |
| Decision Tree Regression |     6.85 |   2.48 |   0.21 |

## Sub-task 2: Include the features in the past

Now, let's consider using the sliding window approach to include the past in the regression model as well. Please have a look at the demo notebook, run your experiment, and report the best models you could have if you apply the sliding window approach.

In [10]:
threshold = 1
window_size = 50
sample_step = 1
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

#all_model_results = run_all_models(df_data=df_data_smoothing, models=models, feature_list=selected_features, threshold=threshold, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, single_run_result=False)

all_model_results = run_all_models(df_data=df_data_smoothing, models=models, feature_list=selected_features, threshold=threshold, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, single_run_result=False)

Running model: Linear Regression
Model for predicting temperature of motor 1:
Model for motor 1:
   Max error      RMSE  Exceed boundary rate
0   0.069482  0.007837                   0.0
1   0.056229  0.008682                   0.0
2   0.072082  0.010125                   0.0
3   0.056745  0.008736                   0.0
4   0.069449  0.010882                   0.0


Mean performance metric and standard error:
Max error: 0.0648 +- 0.0077
RMSE: 0.0093 +- 0.0012
Exceed boundary rate: 0.0000 +- 0.0000


Model for predicting temperature of motor 2:
Model for motor 2:
   Max error      RMSE  Exceed boundary rate
0   0.064611  0.007691                   0.0
1   0.783062  0.037335                   0.0
2   0.083560  0.008195                   0.0
3   0.066895  0.008229                   0.0
4   0.062887  0.008616                   0.0


Mean performance metric and standard error:
Max error: 0.2122 +- 0.3192
RMSE: 0.0140 +- 0.0130
Exceed boundary rate: 0.0000 +- 0.0000


Model for predicting te

In [ ]:
summary_data = {
    'Model': [],
    'Max error': [],
    'RMSE': [],
    'Exceed boundary rate': []
}

for model_name, df_results in all_model_results:
    max_error = df_results['Max error'].mean()
    mse = df_results['RMSE'].mean()
    exceed_boundary_rate = df_results['Exceed boundary rate'].mean()
    
    summary_data['Model'].append(model_name)
    summary_data['Max error'].append(f'{max_error:.2f}')
    summary_data['RMSE'].append(f'{mse:.2f}')
    summary_data['Exceed boundary rate'].append(f'{exceed_boundary_rate:.2f}')

summary_df = pd.DataFrame(summary_data)

print("Summary of the results - Only using features from the current moment\n")
print("| Model                   | Max error | RMSE  | Exceed boundary rate |")
print("|-------------------------|-----------|-------|----------------------|")

for model_name, df_results in all_model_results:
    max_error = df_results['Max error'].mean()
    mse = df_results['RMSE'].mean()
    exceed_boundary_rate = df_results['Exceed boundary rate'].mean()
    
    print(f"| {model_name.ljust(24)} | {max_error:8.2f} | {mse:6.2f} | {exceed_boundary_rate:6.2f} |")

best_model = summary_df.loc[summary_df['RMSE'].idxmin()]
print(f"\nThe best model is {best_model['Model']} with a Mean Squared Error (RMSE) of {best_model['RMSE']}.")

Summary of the results - Only using features from the current moment

| Model                   | Max error | RMSE  | Exceed boundary rate |
|-------------------------|-----------|-------|----------------------|
| Linear Regression        |     0.06 |   0.01 |   0.00 |
| Ridge Regression         |     0.26 |   0.05 |   0.00 |
| Lasso Regression         |     5.84 |   2.24 |   0.48 |
| ElasticNet Regression    |     5.63 |   2.19 |   0.48 |
| Decision Tree Regression |     1.67 |   0.44 |   0.10 |

The best model is Linear Regression with a Mean Squared Error (RMSE) of 0.01.


**Summary of the results - Sliding window**

Please add a table in the end, summarying the results from all the models. Please write a few texts to explain what is the best model you got (including the features and preprocessing you did), its performance, and how could you further improve it.

| Model   (threshold = 1; window_size = 50 ;sample_step = 1 ; prediction_lead_time = 1) | Max error | MRSE | Exceed boundary rate |
|---------|----------|-----------|--------|
| Linear Regression        |     0.06 |   0.01 |   0.00 |
| Ridge Regression         |     0.26 |   0.05 |   0.00 |
| Lasso Regression         |     5.84 |   2.24 |   0.48 |
| ElasticNet Regression    |     5.63 |   2.19 |   0.48 |
| Decision Tree Regression |     1.67 |   0.44 |   0.10 |

# Task 2 Fault detection based on regression model

In this exercise, we use the dataset that contains the failure of motor 6 to test the fault detection model based on the regression model trained before. 

[This notebook](demo_FaultDetectReg.ipynb) presents a demonstration of how to use the provided supporting function to develop fault detection model based on the regression model. Please have a look at this notebook, and try to improve the performance of the baseline model.

In [20]:
n_int = 20

# Subfunction for data preprocessing.
def pre_processing(df: pd.DataFrame):
    ''' ### Description
    Preprocess the data:
    - remove outliers
    - add new features about the difference between the current and previous n data point.
    '''
    
    # Function to design a Butterworth low-pass filter
    def butter_lowpass(cutoff, fs, order=5):
        nyquist = 0.5 * fs
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a


    # Function to apply the Butterworth low-pass filter
    def lowpass_filter(data, cutoff_freq, sampling_freq, order=5):
        b, a = butter_lowpass(cutoff_freq, sampling_freq, order=order)
        filtered_data = filtfilt(b, a, data)
        return filtered_data


    # Set parameters for the low-pass filter
    cutoff_frequency = .8  # Adjust as needed
    sampling_frequency = 10  # Assuming your data is evenly spaced in time


    def customized_outlier_removal(df: pd.DataFrame):
        ''' # Description
        Remove outliers from the dataframe based on defined valid ranges. 
        Define a valid range of temperature and voltage. 
        Use ffil function to replace the invalid measurement with the previous value.
        '''
        df['position'] = df['position'].where(df['position'] <= 1000, np.nan)
        df['position'] = df['position'].where(df['position'] >= 0, np.nan)
        df['position'] = df['position'].ffill()
        df['position'] = lowpass_filter(df['position'], cutoff_frequency, sampling_frequency)
        df['position'] = df['position'].rolling(window=20, min_periods=1).mean()
        df['position'] = df['position'].round()
        df['position'] = df['position'] - df['position'].iloc[0]


        df['temperature'] = df['temperature'].where(df['temperature'] <= 100, np.nan)
        df['temperature'] = df['temperature'].where(df['temperature'] >= 0, np.nan)
        df['temperature'] = df['temperature'].rolling(window=20, min_periods=1).mean()

        # Make sure that the difference between the current and previous temperature cannot be too large.
        # Define your threshold
        threshold = 5
        # Shift the 'temperature' column by one row to get the previous temperature
        prev_tmp = df['temperature'].shift(1)
        # Calculate the absolute difference between current and previous temperature
        temp_diff = np.abs(df['temperature'] - prev_tmp)
        # Set the temperature to NaN where the difference is larger than the threshold
        df.loc[temp_diff > threshold, 'temperature'] = np.nan
        df['temperature'] = df['temperature'].ffill()
        df['temperature'] = df['temperature'] - df['temperature'].iloc[0]


        df['voltage'] = df['voltage'].where(df['voltage'] >= 6000, np.nan)
        df['voltage'] = df['voltage'].where(df['voltage'] <= 8000, np.nan)
        df['voltage'] = df['voltage'].ffill()
        df['voltage'] = lowpass_filter(df['voltage'], cutoff_frequency, sampling_frequency)
        df['voltage'] = df['voltage'].rolling(window=5, min_periods=1).mean()
        df['voltage'] = df['voltage'] - df['voltage'].iloc[0]



    def cal_diff(df: pd.DataFrame, n_int: int):
        ''' # Description
        Calculate the difference between the current and previous n data point.
        '''
        # Tranform the features relative to the first data point.

        # Calculate the difference between the current and previous n data point.
        df['temperature_diff'] = df['temperature'].diff(n_int)
        df['voltage_diff'] = df['voltage'].diff(n_int)
        df['position_diff'] = df['position'].diff(n_int)   

    # Start processing.
    customized_outlier_removal(df)
    #cal_diff(df, n_int)


label_columns = ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label', 'data_motor_5_label', 'data_motor_6_label']
    
# Read all the training dataset.
base_dictionary = '../../dataset/training_data/'
df_data = read_all_test_data_from_path(base_dictionary, pre_processing, is_plot=False)

# Smooth the data.
smoothed_data = df_data.copy(deep=True)
#smoothed_data.drop(columns=['time','test_condition'], inplace=True)

for i in range(1,7):
    smoothed_data[f'data_motor_{i}_voltage'] = smooth_data_moving_average(smoothed_data[f'data_motor_{i}_voltage'], 10)
    

base_dictionary = '../../dataset/testing_data/'
# Get all the folders in the base_dictionary
path_list = os.listdir(base_dictionary)
# Only keep the folders, not the excel file.
path_list_sorted = sorted(path_list)
path_list = path_list_sorted[:-1]

# Read the data.
df_test = pd.DataFrame()
for tmp_path in path_list:
    path = base_dictionary + tmp_path
    #tmp_df = read_all_csvs_one_test(path, tmp_path, pre_processing)
    
    ### ------------read_all_csvs_one_test --------------
    
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]

    # Create an empty DataFrame to store the combined data
    combined_df = pd.DataFrame()

    # Iterate over the CSV files in the folder
    for file in csv_files:
        # Construct the full path to each CSV file
        file_path = os.path.join(path, file)

        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Drop the time. Will add later.
        df = df.drop(labels=df.columns[0], axis=1)

        # Apply the pre-processing.
        if pre_processing:
            pre_processing(df)

        # Extract the file name (excluding the extension) to use as a prefix
        file_name = os.path.splitext(file)[0]

        # Add a prefix to each column based on the file name
        df = df.add_prefix(f'{file_name}_')

        # Concatenate the current DataFrame with the combined DataFrame
        combined_df = pd.concat([combined_df, df], axis=1)

    # Add time and test condition
    df = pd.read_csv(file_path)
    combined_df = pd.concat([df['time'], combined_df], axis=1)

    # Calculate the time difference since the first row
    time_since_first_row = combined_df['time'] - combined_df['time'].iloc[0]
    # Replace the 'time' column with the time difference
    combined_df['time'] = time_since_first_row

    combined_df.loc[:, 'test_condition'] = tmp_path

    combined_df.drop(columns=label_columns, inplace= True)
    
    # Drop the NaN values, which represents the first n data points in the original dataframe.
    #combined_df.dropna(inplace=True)

    tmp_df = combined_df
    
    ### --------------------------------------------
    
    df_test = pd.concat([df_test, tmp_df])
    df_test = df_test.reset_index(drop=True)

# Read the test conditions
df_test_conditions = pd.read_excel(base_dictionary+'Test conditions.xlsx')

# Smooth the data.

#df_test.drop(columns=['time','test_condition'], inplace=True)

for i in range(1,7):
    df_test[f'data_motor_{i}_voltage'] = smooth_data_moving_average(df_test[f'data_motor_{i}_voltage'], 10)


In [7]:

smoothed_data.describe()

,time,data_motor_1_position,data_motor_1_temperature,data_motor_1_voltage,data_motor_1_label,data_motor_1_temperature_diff,data_motor_1_voltage_diff,data_motor_1_position_diff,data_motor_2_position,data_motor_2_temperature,...,data_motor_5_temperature_diff,data_motor_5_voltage_diff,data_motor_5_position_diff,data_motor_6_position,data_motor_6_temperature,data_motor_6_voltage,data_motor_6_label,data_motor_6_temperature_diff,data_motor_6_voltage_diff,data_motor_6_position_diff
count,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,...,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000,38849.000000
mean,207.394232,8.123195,1.961704,0.531346,0.034724,-0.010102,0.384444,0.069191,0.466344,0.172295,...,-0.015154,0.050295,-0.242117,-52.814616,0.786378,-5.268858,0.049731,0.007959,0.008177,-0.577492
std,177.553825,186.961643,8.838867,166.347210,0.183083,0.349660,76.603870,30.467596,27.544395,1.517066,...,0.315020,29.600230,16.918503,179.632435,2.597270,35.248699,0.217391,0.380449,26.518836,60.306199
min,1.999523,-462.000000,-15.000000,-1038.249110,0.000000,-5.000000,-1134.991667,-429.000000,-117.000000,-4.000000,...,-2.000000,-345.179239,-382.000000,-866.000000,-6.950000,-274.071244,0.000000,-3.400000,-301.814450,-835.000000
25%,56.927198,0.000000,-2.900000,-58.007944,0.000000,0.000000,-14.766572,0.000000,0.000000,0.000000,...,0.000000,-8.402816,0.000000,-39.000000,0.000000,-23.082120,0.000000,0.000000,-8.815883,0.000000
50%,152.300156,0.000000,0.200000,-4.336805,0.000000,0.000000,-0.169988,0.000000,0.000000,0.000000,...,0.000000,-0.022093,0.000000,0.000000,0.000000,-1.479435,0.000000,0.000000,0.051212,0.000000
75%,321.100050,0.000000,7.000000,114.734143,0.000000,0.000000,14.528569,0.000000,0.000000,1.000000,...,0.000000,8.776705,0.000000,0.000000,2.000000,12.453061,0.000000,0.000000,8.921466,0.000000
max,679.819920,547.000000,20.000000,425.388172,1.000000,7.600000,1051.237592,371.000000,199.000000,8.950000,...,4.600000,316.919446,220.000000,505.000000,14.500000,124.385226,1.000000,6.850000,302.404664,684.000000


In [8]:
# Pre-train the model.
# Get all the normal data.
normal_test_id = ['20240105_164214', 
    '20240105_165300', 
    '20240105_165972', 
    '20240320_152031', 
    '20240320_153841', 
    '20240320_155664', 
    '20240321_122650', 
    '20240325_135213',
    '20240325_152902', 
    '20240426_141190', 
    '20240426_141532', 
    '20240426_141602', 
    '20240426_141726', 
    '20240426_141938', 
    '20240426_141980', 
    '20240503_164435']

df_tr = smoothed_data[smoothed_data['test_condition'].isin(normal_test_id)]


# Test data.
test_id = [
    '20240527_094865',
    '20240527_100759',
    '20240527_101627',
    '20240527_102436',
    '20240527_102919',
    '20240527_103311',
    '20240527_103690',
    '20240527_104247'
]
df_test = df_test[df_test['test_condition'].isin(test_id)]

In [54]:
feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature','data_motor_1_voltage',
       'data_motor_1_temperature_diff', 'data_motor_1_voltage_diff','data_motor_1_position_diff', 
       'data_motor_2_position','data_motor_2_temperature', 'data_motor_2_voltage', 
       'data_motor_2_temperature_diff', 'data_motor_2_voltage_diff', 'data_motor_2_position_diff',
       'data_motor_3_position', 'data_motor_3_temperature','data_motor_3_voltage',
       'data_motor_3_temperature_diff', 'data_motor_3_voltage_diff','data_motor_3_position_diff', 
       'data_motor_4_position','data_motor_4_temperature', 'data_motor_4_voltage',
       'data_motor_4_temperature_diff', 'data_motor_4_voltage_diff', 'data_motor_4_position_diff',
       'data_motor_5_position', 'data_motor_5_temperature','data_motor_5_voltage',
       'data_motor_5_temperature_diff', 'data_motor_5_voltage_diff','data_motor_5_position_diff', 
       'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage',
       'data_motor_6_temperature_diff', 'data_motor_6_voltage_diff', 'data_motor_6_position_diff']

feature_list_all_6 = ['time',
                 'data_motor_2_position', 
                'data_motor_3_position', 
                'data_motor_4_position', 'data_motor_3_temperature',
                  'data_motor_6_temperature']

feature_list_all_1 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position',
                'data_motor_4_temperature',
                'data_motor_5_position', 'data_motor_5_temperature', 
                'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage','data_motor_6_temperature_diff', 'data_motor_6_voltage_diff', 'data_motor_6_position_diff']

feature_list_all_2 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']
feature_list_all_3 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']
feature_list_all_4 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']
feature_list_all_5 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']


In [73]:
def cross_validation_and_undersampling(X,y):
    
    # Initialize KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        return X_train, y_train, X_test, y_test
    

def run_all_motors_moi(motor_label, drop_list):
    X = df_tr.drop(columns=label_columns+drop_list)
    y = df_tr[motor_label]
    
    X_train, y_train, X_test , y_test = cross_validation_and_undersampling(X,y)
    
    # Ignore warnings.
    warnings.filterwarnings('ignore')

    # Steps for Linear Regression
    linear_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardization
        ('regressor', LinearRegression())    # Step 2: Linear Regression
    ]

    # Steps for Ridge Regression
    ridge_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardization
        ('regressor', Ridge())               # Step 2: Ridge Regression
    ]

    # Steps for Lasso Regression
    lasso_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardizatio
        ('regressor', Lasso())               # Step 2: Lasso Regression
    ]

    # Steps for ElasticNet Regression
    elasticnet_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardization
        ('regressor', ElasticNet())          # Step 2: ElasticNet Regression
    ]

    # Steps for Decision Tree Regression
    decision_tree_steps = [
        ('regressor', DecisionTreeRegressor())  # Step 2: Decision Tree Regressor
    ]

    # Initialize Pipelines for each model
    mdl_linear_regression = Pipeline(linear_regression_steps)
    mdl_ridge_regression = Pipeline(ridge_regression_steps)
    mdl_lasso_regression = Pipeline(lasso_regression_steps)
    mdl_elasticnet_regression = Pipeline(elasticnet_regression_steps)
    mdl_decision_tree = Pipeline(decision_tree_steps)

    # List of models to be used in GridSearchCV
    models = {
        'Linear Regression': mdl_linear_regression,
        'Ridge Regression': mdl_ridge_regression,
        'Lasso Regression': mdl_lasso_regression,
        'ElasticNet Regression': mdl_elasticnet_regression,
        'Decision Tree Regression': mdl_decision_tree,
    }

    # Dictionary to store model performance metrics
    model_metrics = {}

    # Define hyperparameter grids
    param_grids = {
    'Linear Regression': {},  # No hyperparameters for Linear Regression
    'Ridge Regression': {'regressor__alpha': [0.1, 0.001, 1e-5]},  # Alpha parameter for Ridge Regression
    'Lasso Regression': {'regressor__alpha': [0.1, 0.001, 1e-5]},  # Alpha parameter for Lasso Regression
    'ElasticNet Regression': {'regressor__alpha': [0.1, 0.001, 1e-5], 'regressor__l1_ratio': [0.1, 0.001, 0.000001]},  # Alpha and l1_ratio for ElasticNet Regression
    'Decision Tree Regression': {'regressor__max_depth': [None]}  # Max depth for Decision Tree Regression
    }
    model_predictions = {}

    # Perform cross-validation, hyperparameter tuning, and evaluation
    for model_name, model in models.items():
        
        # Hyperparameter tuning
        grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='f1')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        
        # Evaluate on test set
        y_pred = best_model.predict(X_test)
        
        model_predictions[f'y_pred_{model_name.replace(" ", "_")}'] = y_pred
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # Store metrics in the dictionary
        model_metrics[model_name] = {
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2
        }
        
    # Update the summary table with the model performance metrics
    summary_table = "| Model                    | MSE      | RMSE     | R2     |\n"
    summary_table += "|--------------------------|----------|----------|--------|\n"
    for model_name, metrics in model_metrics.items():
        summary_table += f"| {model_name:25} | {metrics['MSE']:.2f}   | {metrics['RMSE']:.2f}   | {metrics['R2']:.2f}  |\n"

    print(summary_table)        

    return model_predictions

In [78]:
def run_all_motors_validation_moi(motor_label, drop_list):
    
    #X = smoothed_data.drop(columns=label_columns+drop_list).values
    df_tr_x = smoothed_data.drop(columns=label_columns+drop_list)
    feature_list_all = smoothed_data.drop(columns=label_columns+drop_list).columns.tolist()
    
    window_size = 50
    sample_step = 10

    df_tr_y = smoothed_data[motor_label]
    
    # Extract the features.
    #df_tr_x, df_tr_y = extract_selected_feature(df_train, feature_list_all, motor_label, mdl_type='clf')

    # Prepare the training data based on the defined sliding window.
    X_train, y_train = prepare_sliding_window(df_x=df_tr_x, y=df_tr_y, window_size=window_size, sample_step=sample_step, mdl_type='reg')
    
    warnings.filterwarnings('ignore')

    # Steps for Linear Regression
    linear_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardization
        ('regressor', LinearRegression())    # Step 2: Linear Regression
    ]

    # Steps for Ridge Regression
    ridge_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardization
        ('regressor', Ridge())               # Step 2: Ridge Regression
    ]

    # Steps for Lasso Regression
    lasso_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardizatio
        ('regressor', Lasso())               # Step 2: Lasso Regression
    ]

    # Steps for ElasticNet Regression
    elasticnet_regression_steps = [
        ('Normalizer', MinMaxScaler()),  # Step 1: Normalization
        #('Standardizer', StandardScaler),  # Step 1: Standardization
        ('regressor', ElasticNet())          # Step 2: ElasticNet Regression
    ]

    # Steps for Decision Tree Regression
    decision_tree_steps = [
        ('regressor', DecisionTreeRegressor())  # Step 2: Decision Tree Regressor
    ]

    # Initialize Pipelines for each model
    mdl_linear_regression = Pipeline(linear_regression_steps)
    mdl_ridge_regression = Pipeline(ridge_regression_steps)
    mdl_lasso_regression = Pipeline(lasso_regression_steps)
    mdl_elasticnet_regression = Pipeline(elasticnet_regression_steps)
    mdl_decision_tree = Pipeline(decision_tree_steps)

    # List of models to be used in GridSearchCV
    models = {
        'Linear Regression': mdl_linear_regression,
        'Ridge Regression': mdl_ridge_regression,
        'Lasso Regression': mdl_lasso_regression,
        'ElasticNet Regression': mdl_elasticnet_regression,
        'Decision Tree Regression': mdl_decision_tree,
    }

    # Dictionary to store model performance metrics
    model_metrics = {}

    # Define hyperparameter grids
    param_grids = {
    'Linear Regression': {},  # No hyperparameters for Linear Regression
    'Ridge Regression': {'regressor__alpha': [0.1, 0.001, 1e-5]},  # Alpha parameter for Ridge Regression
    'Lasso Regression': {'regressor__alpha': [0.1, 0.001, 1e-5]},  # Alpha parameter for Lasso Regression
    'ElasticNet Regression': {'regressor__alpha': [0.1, 0.001, 1e-5], 'regressor__l1_ratio': [0.1, 0.001, 0.000001]},  # Alpha and l1_ratio for ElasticNet Regression
    'Decision Tree Regression': {'regressor__max_depth': [None]}  # Max depth for Decision Tree Regression
    }
    model_predictions = {}

    best_params = {}
    
    # Perform cross-validation, hyperparameter tuning, and evaluation
    for model_name, model in models.items():
        
        pipeline = Pipeline([ # Step 1 : Normalization
            ('model', model)
        ])
        param_grid = {f'model__{key}': value for key, value in param_grids[model_name].items()}
        
        # Hyperparameter tuning
        grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1')
        #grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='f1')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        best_params[model_name] = grid_search.best_params_
        
        # Evaluate on test set
        #feature_list_all.append('test_condition')
        df_test_x = df_test[feature_list_all]
        df_test_y = df_test[['data_motor_1_temperature']]
        #X_test = df_test.drop(columns=drop_list).values
        #df_test_x = df_test.drop(columns=drop_list)
        X_test,y_test = prepare_sliding_window(df_x=df_test_x,y=df_test_y, window_size=window_size, sample_step=sample_step, mdl_type='reg')
    
        y_pred = best_model.predict(X_test)
        
        model_predictions[f'y_pred_{model_name.replace(" ", "_")}'] = y_pred    
    
    return model_predictions, best_params,y_pred

exemple moteur 1

In [57]:
drop_list1_label1 = []
drop_list1_label2 = []
drop_list1_label3 = []
drop_list1_label4 = []
drop_list1_label5 = []
drop_list1_label6 = []

In [79]:
model_predictions = run_all_motors_validation_moi('data_motor_1_label',drop_list1_label1)
'''model_predictions_2 = run_all_motors_validation_moi('data_motor_1_label',drop_list1_label2)
model_predictions_3= run_all_motors_validation_moi('data_motor_1_label',drop_list1_label3)
model_predictions_4 = run_all_motors_validation_moi('data_motor_1_label',drop_list1_label4)
model_predictions_5 = run_all_motors_validation_moi('data_motor_1_label',drop_list1_label5)
model_predictions_6 = run_all_motors_validation_moi('data_motor_1_label',drop_list1_label6)'''

TypeError: 'NoneType' object is not subscriptable

In [81]:
df_test_x = df_test[feature_list_all]
df_test_y = df_test[['data_motor_1_temperature']]
# Prepare feature and response of the training dataset.
x_tr_org, y_temp_tr_org = extract_selected_feature(df_data=df_tr, feature_list=feature_list_all, motor_idx=1, mdl_type='reg')

# Enrich the features based on the sliding window.
window_size = 10
sample_step = 1
prediction_lead_time = 1 
threshold = .9
abnormal_limit = 3

x_tr, y_temp_tr = prepare_sliding_window(df_x=x_tr_org, y=y_temp_tr_org, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, mdl_type='reg')

# Define the steps of the pipeline
steps = [
    ('standardizer', MinMaxScaler()),  # Step 1: StandardScaler
    ('regressor', LinearRegression())    # Step 2: Linear Regression
]

# Create the pipeline
mdl_linear_regreession = Pipeline(steps)
# Fit the model
mdl = mdl_linear_regreession.fit(x_tr, y_temp_tr)

detector_reg = FaultDetectReg(reg_mdl=mdl, threshold=threshold, abnormal_limit=abnormal_limit, window_size=window_size, sample_step=sample_step, pred_lead_time=prediction_lead_time)

_, y_label_test_org = extract_selected_feature(df_data=df_test, feature_list=feature_list_all, motor_idx=1, mdl_type='clf')
x_test_org, y_temp_test_org = extract_selected_feature(df_data=df_test, feature_list=feature_list_all, motor_idx=1, mdl_type='reg')

X_test, y_test_temp = prepare_sliding_window(df_x=df_test_x, y=df_test_y, window_size=window_size, sample_step=sample_step, mdl_type='reg')
y_pred, y_response_test_pred = detector_reg.predict(df_x_test=X_test, y_response_test=y_test_temp)

mse = mean_squared_error(df_test_y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(df_test_y, y_pred)
print('mse = ',mse,'rmse=',rmse,'r2',r2)

KeyError: 'data_motor_1_label'

In [48]:
def evaluate_model(name, steps,param_grid, features, window_size, sample_step, prediction_lead_time, threshold, abnormal_limit, n_fold=8):
    print(f'Running model: {name}')
    
    x_tr_org, y_temp_tr_org = extract_selected_feature(df_data=df_tr, feature_list=features, motor_idx=1, mdl_type='reg')
    
    x_tr, y_temp_tr = prepare_sliding_window(df_x=x_tr_org, y=y_temp_tr_org, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, mdl_type='reg')
    
    
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, param_grid, cv=n_fold, scoring='f1', n_jobs=-1)
    
    grid_search.fit(x_tr, y_temp_tr)
    best_model = grid_search.best_estimator_
    
    #mdl = Pipeline(steps).fit(x_tr, y_temp_tr)
    
    # Define the fault detector
    detector_reg = FaultDetectReg(reg_mdl=best_model, threshold=threshold, abnormal_limit=abnormal_limit, window_size=window_size, sample_step=sample_step, pred_lead_time=prediction_lead_time)
    
    # # Run cross validation
    n_fold = 8
    _, y_label_test_org = extract_selected_feature(df_data=df_test, feature_list=features, motor_idx=1, mdl_type='clf')
    x_test_org, y_temp_test_org = extract_selected_feature(df_data=df_test, feature_list=features, motor_idx=1, mdl_type='reg')
    
    print(f'Best parameters for {name}: {grid_search.best_params_}')
    
    motor_idx = 1
    print(f'Model for motor {motor_idx}:')
    
    # Run cross validation.
    df_perf = detector_reg.run_cross_val(df_x=x_test_org, y_label=y_label_test_org, y_response=y_temp_test_org, 
                                        n_fold=n_fold,single_run_result=False)
    
    print(f'{name} performance:\n{df_perf}\n')
    print('Mean performance metric and standard error:')
    for metric, error in zip(df_perf.mean(), df_perf.std()):
        print(f'{metric:.4f} +- {error:.4f}')
    print('\n')
    return df_perf

In [76]:
# Enrich the features based on the sliding window.
window_size = 100
sample_step = 80
prediction_lead_time = 5 
threshold = .5
abnormal_limit = 3

#param_grids = {
    #'Linear Regression': [{}],  # Note: GridSearchCV requires at least one parameter grid
    #'Ridge Regression': [{'regressor__alpha': np.logspace(-7, 1, num=100)}],# always choose the lowest value
    #'Lasso Regression': [{'regressor__alpha': np.logspace(-7, 1, num=100)}],# always choose the lowest value
    #'ElasticNet Regression': [{'regressor__alpha': np.logspace(-7, 1, num=100), 'regressor__l1_ratio': np.logspace(-7, 1, num=100)}], # always choose the lowest value
    #'Decision Tree Regression': [{'regressor__max_depth': [2,3,4], 'regressor__min_samples_split': [2,3,4]}] # always choose the lowest value
#}

#fix parameters to run faster, since we already know the best chosen parameters

param_grids = {
    'Linear Regression': [{}],  # Note: GridSearchCV requires at least one parameter grid
    'Ridge Regression': [{'regressor__alpha': [0.00001]}],# always choose the lowest value
    'Lasso Regression': [{'regressor__alpha': [0.00001]}],# always choose the lowest value
    'ElasticNet Regression': [{'regressor__alpha': [0.00001], 'regressor__l1_ratio': [0.00001]}], # always choose the lowest value
    'Decision Tree Regression': [{'regressor__max_depth': [2], 'regressor__min_samples_split': [2]}] # always choose the lowest value
}

results = []
models = [
    ('Linear Regression', [('Normalizer', MinMaxScaler()), ('regressor', LinearRegression())]),
    ('Ridge Regression', [('Normalizer', MinMaxScaler()), ('regressor', Ridge())]),
    ('Lasso Regression', [('Normalizer', MinMaxScaler()), ('regressor', Lasso())]),
    ('ElasticNet Regression', [('Normalizer', MinMaxScaler()), ('regressor', ElasticNet())]),
    ('Decision Tree Regression', [('regressor', DecisionTreeRegressor())])
]

for model_name, model_steps in models:
    param_grid = param_grids[model_name]
    df_perf = evaluate_model(model_name, model_steps,param_grid, feature_list_all, window_size, sample_step, prediction_lead_time, threshold, abnormal_limit)
    mean_perf = df_perf.mean()
    results.append((model_name, mean_perf['Accuracy'], mean_perf['Precision'], mean_perf['Recall'], mean_perf['F1 score']))

table_md = "| Model   | Accuracy | Precision | Recall | F1   |\n"
table_md += "|---------|----------|-----------|--------|------|\n"
for result in results:
    model_name, accuracy, precision, recall, f1 = result
    table_md += f"| {model_name} | {accuracy:.2f} | {precision:.2f} | {recall:.2f} | {f1:.2f} |\n"

print(table_md)

Running model: Linear Regression


KeyError: 'data_motor_1_label'

## Summary of the results

Please add a table in the end, summarying the results from all the models (including the unsupervised learning models). Please write a few texts to explain what is the best model you got (including key parameters like threshold, window_size, sample_step, prediction_lead_time, etc), its performance, and how could you further improve it.

| Model   | Accuracy | Precision | Recall | F1   |
|---------|----------|-----------|--------|------|
| Linear Regression | 0.83 | 0.75 | 0.70 | 0.71 |
| Ridge Regression | 0.83 | 0.75 | 0.70 | 0.71 |
| Lasso Regression | 0.82 | 0.73 | 0.68 | 0.69 |
| ElasticNet Regression | 0.82 | 0.73 | 0.68 | 0.69 |
| Decision Tree Regression | 0.72 | 0.71 | 0.73 | 0.64 |




- Best model : Linear Regression

- window_size = 80

- sample_step = 60

- prediction_lead_time = 5 

- threshold = .5

- abnormal_limit = 2

In [9]:
df_test.describe()

,time,data_motor_1_position,data_motor_1_temperature,data_motor_1_voltage,data_motor_1_temperature_diff,data_motor_1_voltage_diff,data_motor_1_position_diff,data_motor_2_position,data_motor_2_temperature,data_motor_2_voltage,...,data_motor_5_voltage,data_motor_5_temperature_diff,data_motor_5_voltage_diff,data_motor_5_position_diff,data_motor_6_position,data_motor_6_temperature,data_motor_6_voltage,data_motor_6_temperature_diff,data_motor_6_voltage_diff,data_motor_6_position_diff
count,14157.000000,14157.000000,14157.000000,14157.000000,13997.000000,13997.000000,13997.000000,14157.000000,14157.000000,14157.000000,...,14157.000000,13997.000000,13997.000000,13997.000000,14157.000000,14157.000000,14157.000000,13997.000000,13997.000000,13997.000000
mean,107.817252,74.411175,-0.515095,-1.131676,-0.013210,0.171651,0.611774,27.124179,-0.419713,-2.250729,...,5.057013,-0.020290,0.105528,-0.177538,25.389065,0.473088,1.281001,0.001429,0.087748,-0.110952
std,76.330896,160.996849,4.069394,115.212449,0.342573,69.061135,39.429568,112.016518,1.813288,111.298561,...,40.356460,0.522239,34.488045,11.982368,242.993699,0.984657,35.899665,0.464280,30.485247,52.505566
min,0.000000,-149.000000,-13.000000,-1005.262446,-1.500000,-985.145113,-455.000000,-262.000000,-5.000000,-1095.603361,...,-279.308787,-3.400000,-285.498250,-104.000000,-609.000000,-1.050000,-227.422176,-5.000000,-228.206424,-565.000000
25%,44.629690,0.000000,-1.000000,-46.313697,0.000000,-16.035954,0.000000,-1.000000,-1.000000,-52.050080,...,-8.322519,0.000000,-8.396322,0.000000,0.000000,0.000000,-17.653770,0.000000,-8.798547,0.000000
50%,94.214740,0.000000,0.000000,2.770504,0.000000,0.150923,0.000000,0.000000,0.000000,-4.592442,...,3.448634,0.000000,0.000909,0.000000,0.000000,0.000000,5.244208,0.000000,0.207029,0.000000
75%,161.014850,56.000000,1.000000,47.637558,0.000000,16.144061,0.000000,0.000000,0.000000,34.655231,...,17.358102,0.000000,8.345737,0.000000,1.000000,1.000000,18.191908,0.000000,8.945550,0.000000
max,320.778320,494.000000,13.900000,276.843129,1.700000,867.205015,402.000000,482.000000,11.700000,252.152380,...,113.201205,7.500000,300.202822,134.000000,773.000000,8.200000,95.599813,6.900000,232.677539,549.000000


In [22]:
from utility import extract_selected_feature, prepare_sliding_window
# Pre-train the model.
# Get all the normal data.
normal_test_id = ['20240105_164214', 
    '20240105_165300', 
    '20240105_165972', 
    '20240320_152031', 
    '20240320_153841', 
    '20240320_155664', 
    '20240321_122650', 
    '20240325_135213',
    '20240325_152902', 
    '20240426_141190', 
    '20240426_141532', 
    '20240426_141602', 
    '20240426_141726', 
    '20240426_141938', 
    '20240426_141980', 
    '20240503_164435']

df_train = smoothed_data[smoothed_data['test_condition'].isin(normal_test_id)]


# Test data.
test_id = [
    '20240527_094865',
    '20240527_100759',
    '20240527_101627',
    '20240527_102436',
    '20240527_102919',
    '20240527_103311',
    '20240527_103690',
    '20240527_104247'
]
df_test = df_test[df_test['test_condition'].isin(test_id)]
# Define the features.
feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature','data_motor_1_voltage',
       'data_motor_2_position','data_motor_2_temperature', 'data_motor_2_voltage', 
       'data_motor_3_position', 'data_motor_3_temperature','data_motor_3_voltage',
       'data_motor_4_position','data_motor_4_temperature', 'data_motor_4_voltage',
       'data_motor_5_position', 'data_motor_5_temperature','data_motor_5_voltage',
       'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage'
       ]

feature_list_all_6 = ['time',
                 'data_motor_2_position', 
                'data_motor_3_position', 
                'data_motor_4_position', 'data_motor_3_temperature',
                  'data_motor_6_temperature']

feature_list_all_1 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position',
                'data_motor_4_temperature',
                'data_motor_5_position', 'data_motor_5_temperature', 
                'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage','data_motor_6_temperature_diff', 'data_motor_6_voltage_diff', 'data_motor_6_position_diff']

feature_list_all_2 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']

feature_list_all_3 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']

feature_list_all_4 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']

feature_list_all_5 = ['time','data_motor_1_position', 'data_motor_1_temperature',
                'data_motor_2_position', 'data_motor_2_temperature',
                'data_motor_3_position','data_motor_3_temperature',
                'data_motor_4_temperature','data_motor_4_position',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_temperature', 'data_motor_6_voltage']# Extract the features.

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from utility import read_all_test_data_from_path, extract_selected_feature, prepare_sliding_window, FaultDetectReg

# Pre-train the model.
# Get all the normal data for motor 5.
normal_test_id = ['20240105_164214',
    '20240105_165300',
    '20240105_165972',
    '20240320_152031',
    '20240320_153841',
    '20240320_155664',
    '20240321_122650',
    '20240325_135213',
    '20240325_152902',
    '20240426_141190',
    '20240426_141532',
    '20240426_141602',
    '20240426_141726',
    '20240426_141938',
    '20240426_141980',
    '20240503_163963',
    '20240503_164435',
    '20240503_164675',
    '20240503_165189'
]

df_train = df_train.fillna(df_train.drop(columns=['test_condition']).median())
df_tr_motor_5 = df_train[df_train['test_condition'].isin(normal_test_id)]


# Prepare feature and response of the training dataset.
x_tr_org, y_temp_tr_org = extract_selected_feature(df_data=df_tr_motor_5, feature_list=feature_list_all, motor_idx=5, mdl_type='reg')

# Enrich the features based on the sliding window.
window_size = 80
sample_step = 60
prediction_lead_time = 5
threshold = .5
abnormal_limit = 3

x_tr, y_temp_tr = prepare_sliding_window(df_x=x_tr_org, y=y_temp_tr_org, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, mdl_type='reg')

# Define the steps of the pipeline
steps = [
    ('standardizer', MinMaxScaler()),  # Step 1: StandardScaler
    ('regressor', LinearRegression())    # Step 2: Linear Regression
]

# Create the pipeline
mdl_linear_regreession = Pipeline(steps)
# Fit the model
mdl = mdl_linear_regreession.fit(x_tr, y_temp_tr)


# Define the fault detector.
detector_reg = FaultDetectReg(reg_mdl=mdl, threshold=threshold, abnormal_limit=abnormal_limit, window_size=window_size, sample_step=sample_step, pred_lead_time=prediction_lead_time)

df_test = df_test.fillna(df_test.drop(columns=['test_condition']).median())# Prepare the testing data.
x_test_org, y_temp_test_org = extract_selected_feature(df_data=df_test, feature_list=feature_list_all, motor_idx=5, mdl_type='reg')

# Make predicition.
y_pred_5, y_response_test_pred = detector_reg.predict(df_x_test=x_test_org, y_response_test=y_temp_test_org)


import pandas as pd

# Read the template.
path = '../../ws_group_1/deliverables/sample_submission.csv' # Change to your path.
df_submission = pd.read_csv(path)

# Initial all values with -1.
df_submission.loc[:, ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label', 'data_motor_5_label', 'data_motor_6_label']] = -1
df_submission.head()

# Replace each column with your prediction.
df_submission['data_motor_5_label'] = y_pred_5


# For the other motors, we just fill with 0.
df_submission.loc[:, ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label']] = 0

# Output the submission csv.
df_submission.to_csv('../../ws_group_1/deliverables/submission.csv')



100%|██████████| 8/8 [00:23<00:00,  2.98s/it]


ValueError: Length of values (13525) does not match length of index (14157)